In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001

    # 데이터 준비
    dataset = EmotionDataset(data_dir=DATA_DIR)
    # 실제 클래스 수를 데이터셋에서 가져와 업데이트
    NUM_CLASSES = len(dataset.classes)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, dataloader, criterion, optimizer, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

데이터 준비 완료!
훈련 데이터셋 크기: 2210
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'resnet18' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/10
----------
  Batch 10/139 Loss: 2.2997
  Batch 20/139 Loss: 1.8917
  Batch 30/139 Loss: 1.9244
  Batch 40/139 Loss: 1.7704
  Batch 50/139 Loss: 1.8724
  Batch 60/139 Loss: 2.2426
  Batch 70/139 Loss: 1.9314
  Batch 80/139 Loss: 1.7778
  Batch 90/139 Loss: 1.8801
  Batch 100/139 Loss: 2.2224
  Batch 110/139 Loss: 1.9506
  Batch 120/139 Loss: 2.0037
  Batch 130/139 Loss: 2.0805
Train Loss: 1.9858 Acc: 0.2054

Epoch 2/10
----------
  Batch 10/139 Loss: 1.7440
  Batch 20/139 Loss: 2.0319
  Batch 30/139 Loss: 1.7554
  Batch 40/139 Loss: 2.0765
  Batch 50/139 Loss: 1.9001
  Batch 60/139 Loss: 1.8847
  Batch 70/139 Loss: 1.8932
  Batch 80/139 Loss: 2.0150
  Batch 90/139 Loss: 1.6017
  Batch 100/139 Loss: 1.8648
  Batch 110/139 Loss: 1.8648
  Batch 120/139 Loss: 1.6670
  Batch 130/139 Loss: 1.8681
Train Loss: 1.8417 Acc: 0.2566

Epoch 3/10
----------

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent_preprocessed")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    
    # 훈련용과 검증용 데이터셋을 각각 생성합니다.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train")
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val")
    
    # 데이터로더를 각각 생성합니다. (검증용은 섞을 필요가 없습니다)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, DEVICE, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cpu


ValueError: num_samples should be a positive integer value, but got num_samples=0